# PC Algorithm based MonitorRank

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from collections import defaultdict, OrderedDict

import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', FutureWarning)

import sys
sys.path.append("../")

from notebooklib.save import run_tsdr, save_tsdr, load_tsdr, load_tsdr_by_chaos

In [4]:
DATASET_ID = "9n6mf"

In [5]:
from meltria.priorknowledge.priorknowledge import SockShopKnowledge

pk = SockShopKnowledge(
    target_metric_types={
        "containers": True,
        "services": True,
        "middlewares": True,
        "nodes": False,
    },
    mappings={"nodes-containers": {}},
)

In [5]:
import functools
import multiprocessing

from diagnoser.diag import prepare_init_graph, fix_edge_directions_in_causal_graph, find_connected_subgraphs
from eval.groundtruth import check_cause_metrics
import diagnoser.metric_node as mn

# https://github.com/joblib/joblib/pull/366#issuecomment-267603530
def with_timeout(timeout):
    def decorator(decorated):
        @functools.wraps(decorated)
        def inner(*args, **kwargs):
            pool = multiprocessing.pool.ThreadPool(1)
            async_result = pool.apply_async(decorated, args, kwargs)
            try:
                return async_result.get(timeout)
            except multiprocessing.TimeoutError:
                record = args[1]
                print(f"Timeouted {record.chaos_case_full()}")
                pool.terminate()
                return
        return inner
    return decorator

# @with_timeout(10*60)  # 10 min timeout

## Overall evaluation

In [27]:
datasets = load_tsdr_by_chaos(
    DATASET_ID, suffix="fluxrank_pearsonr_medoid_only_ctnrs", revert_normalized_time_series=True,
)

In [28]:
from diagnoser import diag
from notebooklib import rank

In [29]:
import joblib

@with_timeout(10*60)  # 10 min timeout
def diagnose_and_rank(reduced_df, record, **kwargs):
    G, ranks = diag.build_and_walk_causal_graph(
        reduced_df, record.pk,
        enable_prior_knowledge=False,
        pc_library="cdt",
        cg_algo="pc",
        pc_citest_alpha=0.05,
        pc_citest="gaussian",
        pc_variant="stable",
        disable_orientation=False,
        disable_ci_edge_cut=False,
        walk_method="monitorrank",
        root_metric_type="latency",
    )
    if len(ranks) == 0:
        print(f"Failed to diagnose {record.chaos_case_full()} with {len(ranks)} ranks")
        return None
    return G, rank.create_rank_as_dataframe(ranks, DATASET_ID, record)

ranked_datasets = []
for (_, _), somethings_records in datasets.items():
    records = []
    for record, data_df_by_metric_type in somethings_records:
        reduced_df = pd.concat([
            data_df_by_metric_type["services"][-1],
            data_df_by_metric_type["containers"][-1],
            data_df_by_metric_type["middlewares"][-1],
        ], axis=1)
        records.append((record, reduced_df))

    results = joblib.Parallel(n_jobs=-1)(
        joblib.delayed(diagnose_and_rank)(reduced_df, record)
        for record, reduced_df in records
    )
    assert results is not None
    ranked_datasets.extend(result for result in results if result is not None)
ranked_datasets = [_[1] for _ in ranked_datasets]

In [30]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    ranks_df = rank.create_rank_as_dataframe_for_multiple_cases_from_frames(ranked_datasets)
    display(ranks_df)

metric_name  \
dataset_id target_app chaos_type          chaos_comp   chaos_idx                                                   
9n6mf      sock-shop  pod-network-latency user-db      4                            c-payment_memory_usage_bytes   
                                                       4                   c-catalogue-db_cpu_user_seconds_total   
                                                       4               c-front-end_network_receive_packets_total   
                                                       4              c-front-end_network_transmit_packets_total   
                                                       4             c-front-end_cpu_cfs_throttled_seconds_total   
                                                       4                c-catalogue_network_transmit_bytes_total   
                                                       4                                      s-shipping_latency   
                                                       4                 c-front-end_network_receive_bytes_total   
                                                       4                         c-carts-db_fs_reads_bytes_total   
                                                       4                                     c-orders_memory_rss   
                                                       3                                         s-carts_latency   
                                                       3                                  s-catalogue_throughput   
                                                       3                                     s-catalogue_latency   
                                                       3                                          s-user_latency   
                                                       3                 c-payment_network_receive_packets_total   
                                                       3                                       s-user_throughput   
                                                       3                                  s-front-end_throughput   
                                                       3                                      s-shipping_latency   
                                                       3                                        s-orders_latency   
                                                       3                       c-shipping_cpu_user_seconds_total   
                                                       2                                     s-catalogue_latency   
                                                       2                 c-payment_network_receive_packets_total   
                                                       2                               c-user_memory_usage_bytes   
                                                       2                      c-shipping_cpu_usage_seconds_total   
                                                       2                  c-carts-db_network_receive_bytes_total   
                                                       2                          c-user_cpu_usage_seconds_total   
                                                       2                     c-carts-db_cpu_system_seconds_total   
                                                       2                                     s-front-end_latency   
                                                       2                 c-orders-db_network_receive_bytes_total   
                                                       2                        c-carts-db_fs_writes_bytes_total   
                                                       1                c-catalogue_network_transmit_bytes_total   
                                                       1                                     s-catalogue_latency   
                                                       1                   c-payment_network_receive_bytes_total   
                                         

In [31]:
rank.create_localization_score_as_dataframe(ranked_datasets, pk, k=10)

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, payment
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf,

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,30,0.233333,0.233333,110,0.127273,0.127273,110,0.154545,0.154545
2,30,0.433333,0.333333,110,0.272727,0.200000,110,0.336364,0.245455
3,30,0.666667,0.444444,110,0.354545,0.251515,110,0.581818,0.357576
4,30,0.766667,0.525000,110,0.445455,0.300000,110,0.763636,0.459091
5,30,0.800000,0.580000,110,0.463636,0.332727,110,0.827273,0.532727
6,30,0.800000,0.616667,110,0.463636,0.354545,110,0.827273,0.581818
7,30,0.900000,0.657143,110,0.463636,0.370130,110,0.827273,0.616883
8,30,1.000000,0.700000,110,0.463636,0.381818,110,0.827273,0.643182
9,30,1.000000,0.733333,110,0.463636,0.390909,110,0.827273,0.663636
10,30,1.000000,0.760000,110,0.463636,0.398182,110,0.827273,0.680000


In [32]:
rank.create_localization_score_as_dataframe(ranked_datasets, pk, k=10, group_by_cause_type=True)

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, payment
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf,

#cases (metric)  AC@K (metric)  \
9n6mf sock-shop pod-cpu-hog         1                12       0.583333   
                                    2                12       0.666667   
                                    3                12       0.916667   
                                    4                12       1.000000   
                                    5                12       1.000000   
                                    6                12       1.000000   
                                    7                12       1.000000   
                                    8                12       1.000000   
                                    9                12       1.000000   
                                    10               12       1.000000   
                pod-memory-hog      1                11       0.000000   
                                    2                11       0.272727   
                                    3                11       0.545455   
                                    4                11       0.727273   
                                    5                11       0.727273   
                                    6                11       0.727273   
                                    7                11       0.909091   
                                    8                11       1.000000   
                                    9                11       1.000000   
                                    10               11       1.000000   
                pod-network-latency 1                 7       0.000000   
                                    2                 7       0.285714   
                                    3                 7       0.428571   
                                    4                 7       0.428571   
                                    5                 7       0.571429   
                                    6                 7       0.571429   
                                    7                 7       0.714286   
                                    8                 7       1.000000   
                                    9                 7       1.000000   
                                    10                7       1.000000   

                                        AVG@K (metric)  #cases (container)  \
9n6mf sock-shop pod-cpu-hog         1         0.583333                  32   
                                    2         0.625000                  32   
                                    3         0.722222                  32   
                                    4         0.791667                  32   
                                    5         0.833333                  32   
                                    6         0.861111                  32   
                                    7         0.880952                  32   
                                    8         0.895833                  32   
                                    9         0.907407                  32   
                                    10        0.916667                  32   
                pod-memory-hog      1         0.000000                  42   
                                    2         0.136364                  42   
                                    3         0.272727                  42   
                                    4         0.386364                  42   
                                    5         0.454545                  42   
                                    6         0.500000                  42   
                                    7         0.558442                  42   
                                    8         0.613636                  42   
                                    9         0.656566                  42   
                                    10        0.690909                  42   
                pod-network-latency 1         0.000000                  36   
                             

In [33]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rank.create_localization_score_as_dataframe(ranked_datasets, pk, k=10, group_by_cause_type=False, group_by_cause_comp=True))

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, 

#cases (metric)  AC@K (metric)  \
9n6mf sock-shop carts        1                 2       0.000000   
                             2                 2       0.500000   
                             3                 2       1.000000   
                             4                 2       1.000000   
                             5                 2       1.000000   
                             6                 2       1.000000   
                             7                 2       1.000000   
                             8                 2       1.000000   
                             9                 2       1.000000   
                             10                2       1.000000   
                carts-db     1                 6       0.166667   
                             2                 6       0.333333   
                             3                 6       0.833333   
                             4                 6       0.833333   
                             5                 6       0.833333   
                             6                 6       0.833333   
                             7                 6       0.833333   
                             8                 6       1.000000   
                             9                 6       1.000000   
                             10                6       1.000000   
                catalogue    1                 3       0.000000   
                             2                 3       0.000000   
                             3                 3       0.666667   
                             4                 3       0.666667   
                             5                 3       0.666667   
                             6                 3       0.666667   
                             7                 3       1.000000   
                             8                 3       1.000000   
                             9                 3       1.000000   
                             10                3       1.000000   
                catalogue-db 1                 3       0.666667   
                             2                 3       1.000000   
                             3                 3       1.000000   
                             4                 3       1.000000   
                             5                 3       1.000000   
                             6                 3       1.000000   
                             7                 3       1.000000   
                             8                 3       1.000000   
                             9                 3       1.000000   
                             10                3       1.000000   
                orders       1                 2       0.500000   
                             2                 2       1.000000   
                             3                 2       1.000000   
                             4                 2       1.000000   
                             5                 2       1.000000   
                             6                 2       1.000000   
                             7                 2       1.000000   
                             8                 2       1.000000   
                             9                 2       1.000000   
                             10                2       1.000000   
                orders-db    1                 2       0.000000   
                             2                 2       0.000000   
                             3                 2       0.000000   
                             4                 2       0.000000   
                             5                 2       0.000000   
                             6                 2       0.000000   
                             7                 2       0.500000   
                             8                 2       1.000000   
                             9                 2       1.000000   
             

In [34]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rank.create_localization_score_as_dataframe(ranked_datasets, pk, k=10, group_by_cause_type=True, group_by_cause_comp=True))

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, 

#cases (metric)  \
9n6mf sock-shop carts        pod-cpu-hog         1                 0   
                                                 2                 0   
                                                 3                 0   
                                                 4                 0   
                                                 5                 0   
                                                 6                 0   
                                                 7                 0   
                                                 8                 0   
                                                 9                 0   
                                                 10                0   
                             pod-memory-hog      1                 2   
                                                 2                 2   
                                                 3                 2   
                                                 4                 2   
                                                 5                 2   
                                                 6                 2   
                                                 7                 2   
                                                 8                 2   
                                                 9                 2   
                                                 10                2   
                             pod-network-latency 1                 0   
                                                 2                 0   
                                                 3                 0   
                                                 4                 0   
                                                 5                 0   
                                                 6                 0   
                                                 7                 0   
                                                 8                 0   
                                                 9                 0   
                                                 10                0   
                carts-db     pod-cpu-hog         1                 2   
                                                 2                 2   
                                                 3                 2   
                                                 4                 2   
                                                 5                 2   
                                                 6                 2   
                                                 7                 2   
                                                 8                 2   
                                                 9                 2   
                                                 10                2   
                             pod-memory-hog      1                 2   
                                                 2                 2   
                                                 3                 2   
                                                 4                 2   
                                                 5                 2   
                                                 6                 2   
                                                 7                 2   
                                                 8                 2   
                                                 9                 2   
                                                 10                2   
                             pod-network-latency 1                 2   
                                                 2                 2   
                                                 3                 2   
                                                 4                 2   
                                                 5                 2   
                     

## Overall Combination Evaluation

In [6]:
from notebooklib import localization

results = localization.grid_dataset({
    "hierarchy-single_sbd_medoid_only_ctnrs",
    "hdbscan_sbd_only_ctnrs",
    "fluxrank_pearsonr_medoid_only_ctnrs",
    "fluxrank_pearsonr_max_cluster_only_ctnrs",
    "fluxrank_sbd_medoid_only_ctnrs",
    "hdbscan_sbd_max_cluster_only_ctnrs",
}, DATASET_ID, n=10, diag_options=dict(
    enable_prior_knowledge=False,
    pc_library="cdt",
    cg_algo="pc",
    pc_citest_alpha=0.05,
    pc_citest="gaussian",
    pc_variant="stable",
    disable_orientation=False,
    disable_ci_edge_cut=False,
    walk_method="monitorrank",
    root_metric_type="latency",
))
for cache_suffix, score_df in results:
    display(cache_suffix, score_df)

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, payment
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, payment
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user-db
no cause metrics: 9n6mf, sock-

adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, payment
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9

adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
adjacency_matrix will return a scipy.sparse

...

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,32,0.34375,0.343750,110,0.145455,0.145455,110,0.018182,0.018182
2,32,0.56250,0.453125,110,0.254545,0.200000,110,0.172727,0.095455
3,32,0.68750,0.531250,110,0.363636,0.254545,110,0.418182,0.203030
4,32,0.78125,0.593750,110,0.445455,0.302273,110,0.663636,0.318182
5,32,0.84375,0.643750,110,0.490909,0.340000,110,0.809091,0.416364
6,32,0.93750,0.692708,110,0.490909,0.365152,110,0.881818,0.493939
7,32,0.96875,0.732143,110,0.490909,0.383117,110,0.900000,0.551948
8,32,0.96875,0.761719,110,0.490909,0.396591,110,0.900000,0.595455
9,32,1.00000,0.788194,110,0.490909,0.407071,110,0.900000,0.629293
10,32,1.00000,0.809375,110,0.490909,0.415455,110,0.900000,0.656364


'fluxrank_pearsonr_medoid_only_ctnrs'

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,30,0.233333,0.233333,110,0.100000,0.100000,110,0.000000,0.000000
2,30,0.400000,0.316667,110,0.245455,0.172727,110,0.190909,0.095455
3,30,0.666667,0.433333,110,0.354545,0.233333,110,0.381818,0.190909
4,30,0.766667,0.516667,110,0.418182,0.279545,110,0.572727,0.286364
5,30,0.800000,0.573333,110,0.436364,0.310909,110,0.727273,0.374545
6,30,0.800000,0.611111,110,0.454545,0.334848,110,0.809091,0.446970
7,30,0.900000,0.652381,110,0.463636,0.353247,110,0.827273,0.501299
8,30,1.000000,0.695833,110,0.463636,0.367045,110,0.827273,0.542045
9,30,1.000000,0.729630,110,0.463636,0.377778,110,0.827273,0.573737
10,30,1.000000,0.756667,110,0.463636,0.386364,110,0.827273,0.599091


'hierarchy-single_sbd_medoid_only_ctnrs'

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,31,0.580645,0.580645,105,0.209524,0.209524,105,0.038095,0.038095
2,31,0.677419,0.629032,105,0.295238,0.252381,105,0.209524,0.123810
3,31,0.741935,0.666667,105,0.409524,0.304762,105,0.428571,0.225397
4,31,0.806452,0.701613,105,0.476190,0.347619,105,0.609524,0.321429
5,31,0.935484,0.748387,105,0.514286,0.380952,105,0.780952,0.413333
6,31,0.967742,0.784946,105,0.514286,0.403175,105,0.847619,0.485714
7,31,1.000000,0.815668,105,0.514286,0.419048,105,0.866667,0.540136
8,31,1.000000,0.838710,105,0.514286,0.430952,105,0.866667,0.580952
9,31,1.000000,0.856631,105,0.514286,0.440212,105,0.866667,0.612698
10,31,1.000000,0.870968,105,0.514286,0.447619,105,0.866667,0.638095
